In [10]:
import csv
import re
import sys
import os
# import tkinter as tk  # No longer needed
# from tkinter import filedialog # No longer needed

# def select_file(title_text, file_types):
#     """
#     開啟檔案選擇視窗
#     """
#     root = tk.Tk()
#     root.withdraw()
#     root.attributes('-topmost', True)

#     file_path = filedialog.askopenfilename(
#         title=title_text,
#         filetypes=file_types
#     )
#     root.destroy()
#     return file_path

def parse_sap_bom(file_path):
    sap_bom = {}

    if not file_path:
        print("SAP 檔案路徑未提供。")
        return {}

    # 嘗試多種編碼讀取 SAP 檔
    lines = []
    encodings = ['utf-8', 'big5', 'gbk', 'utf-16']

    for enc in encodings:
        try:
            with open(file_path, 'r', encoding=enc) as f:
                lines = f.readlines()
            break # 讀取成功就跳出迴圈
        except UnicodeDecodeError:
            continue

    if not lines:
        print(f"錯誤: 無法讀取 SAP 檔案 '{file_path}' (嘗試了 utf-8, big5, gbk, utf-16 都失敗)")
        return {}

    current_part = "UNKNOWN_HEADER"

    part_pattern = re.compile(r'^([A-Z0-9\-]+)\s+(\d+)$')

    desc_keywords = [
        'CAP ', 'RES ', 'DIO ', 'XSTR ', 'PCB ', 'STMP ', 'IND ', 'BEAD ', 'JACK ', 'CONN ',
        'IC ', 'WF+', 'H-SINK', 'COND-HEAT', 'LABEL', 'M/B', 'WIRE CLIP', 'SHIELD',
        'POLYMER', 'VARISTOR', 'SENS', 'LPD4', 'EMMC', 'MODULE', 'BALUN', 'TUNER',
        '豐達興', '慎用', 'TERM', 'LED', 'XTAL', 'XFMR'
    ]

    for line in lines:
        # 去除 標籤
        line = re.sub(r'\\', '', line).strip()

        if not line: continue

        match = part_pattern.match(line)
        if match and '-' in match.group(1) and len(match.group(1)) > 5:
            current_part = match.group(1)
            continue

        if any(k in line for k in desc_keywords):
            continue

        if ',' in line or re.match(r'^[A-Z]+', line):
            refs = [r.strip() for r in line.split(',')]
            for ref in refs:
                clean_ref = ref.replace('"', '').strip()
                if clean_ref:
                    if clean_ref not in sap_bom:
                        sap_bom[clean_ref] = set()
                    sap_bom[clean_ref].add(current_part)

    return sap_bom

def parse_xls_bom(file_path):
    xls_bom = {}

    if not file_path:
        print("XLS/CSV 檔案路徑未提供。")
        return {}

    # 檢查是否不小心選到了 .xls 或 .xlsx (非 CSV)
    if file_path.lower().endswith(('.xls', '.xlsx')):
        print("\n" + "!"*50)
        print("【注意】您選擇了 Excel 檔 (.xls/.xlsx)！")
        print("此程式只能讀取 CSV 檔 (文字格式)。")
        print("請打開 Excel -> 檔案 -> 另存新檔 -> 選擇 CSV (逗號分隔) (*.csv)")
        print("!"*50 + "\n")
        # 這裡不強制 return，因為有時候檔名是 .xls 但內容其實是 csv，我們嘗試讀讀看

    # 嘗試多種編碼，特別是 Big5 (台灣常用)
    encodings_to_try = ['utf-8-sig', 'big5', 'cp950', 'gbk']

    read_success = False

    for enc in encodings_to_try:
        try:
            with open(file_path, 'r', encoding=enc) as f:
                reader = csv.DictReader(f)
                # 測試讀取第一行來確認編碼是否正確
                rows = list(reader)

                # 如果成功讀取到這裡沒報錯，開始處理資料
                for row in rows:
                    # 嘗試抓取欄位，容許一些常見的欄位名稱變化
                    part = row.get('子件') or row.get('Part Number') or row.get('Part')
                    refs_str = row.get('子項目(元件位置)') or row.get('RefDes') or row.get('Location')

                    if not part or not refs_str: continue

                    refs = str(refs_str).split(',')
                    for ref in refs:
                        clean_ref = ref.replace('"', '').strip()
                        if clean_ref:
                            if clean_ref not in xls_bom:
                                xls_bom[clean_ref] = set()
                            xls_bom[clean_ref].add(part)

                read_success = True
                # print(f"成功使用 {enc} 編碼讀取 CSV") # 除錯用
                break # 成功就跳出
        except (UnicodeDecodeError, csv.Error):
            continue # 換下一個編碼試試看

    if not read_success:
        print(f"錯誤: 無法讀取 XLS/CSV 檔案 '{file_path}'。請確認檔案已另存為 'CSV (逗號分隔)' 格式，並嘗試提供正確的檔案路徑。")
        return {}

    return xls_bom

def compare_boms():
    print("--- 請提供檔案路徑 ---")

    # 1. 輸入 SAP 檔案路徑
    sap_file_input = input("請輸入 SAP BOM 檔案 (.txt) 的路徑: ")
    sap_file = sap_file_input.strip().strip('"') # Clean input: remove leading/trailing whitespace and quotes
    if not sap_file:
        print("未提供 SAP 檔案路徑，程式結束。")
        return
    print(f"已輸入 SAP 檔案路徑: {sap_file}")

    sap_data = parse_sap_bom(sap_file)
    if not sap_data:
        print("SAP 檔案讀取失敗，程式結束。")
        return
    print(f"SAP 讀取完成，共 {len(sap_data)} 個位置")

    # 2. 輸入 XLS 轉出的 CSV 檔案路徑
    xls_file_input = input("請輸入 Excel 轉出的 CSV 檔案 (.csv) 的路徑: ")
    xls_file = xls_file_input.strip().strip('"') # Clean input: remove leading/trailing whitespace and quotes
    if not xls_file:
        print("未提供 CSV 檔案路徑，程式結束。")
        return
    print(f"已輸入 CSV 檔案路徑: {xls_file}")

    xls_data = parse_xls_bom(xls_file)
    if not xls_data:
        print("XLS/CSV 檔案讀取失敗，程式結束。")
        return
    print(f"XLS 讀取完成，共 {len(xls_data)} 個位置")

    print("-" * 30)
    print("開始比對...\n")

    all_refs = set(sap_data.keys()).union(set(xls_data.keys()))
    mismatches = []

    for ref in sorted(all_refs):
        sap_parts = sap_data.get(ref, set())
        xls_parts = xls_data.get(ref, set())

        sap_parts_clean = {p for p in sap_parts if p != "UNKNOWN_HEADER"}

        if sap_parts_clean != xls_parts:
            # 忽略因為檔案開頭截斷造成的空白
            if not sap_parts_clean and xls_parts:
                continue
            mismatches.append((ref, sap_parts_clean, xls_parts))

    if not mismatches:
        print("比對結果：完全相符！(Perfect Match)")
    else:
        print(f"比對結果：發現 {len(mismatches)} 個差異")
        print("-" * 60)
        print(f"{'位置 (RefDes)':<15} | {'SAP 料號':<20} | {'XLS 料號':<20}")
        print("-" * 60)
        for ref, sap, xls in mismatches:
            sap_str = ", ".join(map(str, sap)) if sap else "N/A"
            xls_str = ", ".join(map(str, xls)) if xls else "N/A"
            print(f"@{ref:<15} | {sap_str:<20} | {xls_str:<20}")

if __name__ == '__main__':
    compare_boms()

--- 請提供檔案路徑 ---
請輸入 SAP BOM 檔案 (.txt) 的路徑: /SAP.txt
已輸入 SAP 檔案路徑: /SAP.txt
SAP 讀取完成，共 540 個位置
請輸入 Excel 轉出的 CSV 檔案 (.csv) 的路徑: /1319D.csv
已輸入 CSV 檔案路徑: /1319D.csv
XLS 讀取完成，共 541 個位置
------------------------------
開始比對...

比對結果：發現 9 個差異
------------------------------------------------------------
位置 (RefDes)     | SAP 料號               | XLS 料號              
------------------------------------------------------------
@CD34            | 52056-4560G-101      | 52056A4560J         
@CD47            | 52056-4181J-001      | 52056-4181J-001, 52056A4181J
@CM47            | 53111-01203-001      | 53111-01203-002     
@CM48            | 53111-01203-001      | 53111-01203-002     
@CM49            | 53111-01203-001      | 53111-01203-002     
@CM50            | 53111-01203-001      | 53111-01203-002     
@MBN-333         | 41212-D6126-140      | N/A                 
@Y1              | 54101-00755          | 54101-00711         
@YD1             | 54101-00755          | 54101-00711         


## 操作 SOP (Standard Operating Procedure)

### 步驟 1: 準備檔案
1.  **SAP BOM 檔案 (.txt):** 確保您的 SAP BOM 檔案是 `.txt` 格式。
2.  **XLS/CSV 檔案 (.csv):** 如果您有的是 Excel 檔 (.xls 或 .xlsx)，請務必先用 Excel 開啟，然後選擇「另存新檔」，將其儲存為「CSV (逗號分隔) (*.csv)」格式。

### 步驟 2: 上傳檔案到 Colab 環境
由於 Colab 是一個雲端環境，無法直接存取您本機電腦的 D 槽等路徑。您需要將檔案上傳到 Colab:
1.  **在 Colab 介面左側**，點擊「檔案」圖示 (一個資料夾的圖示)。
2.  **將您的 `.txt` 和 `.csv` 檔案**從本機電腦直接拖曳到 Colab 的檔案瀏覽器中，或者點擊「上傳」按鈕。
3.  **上傳完成後**，這些檔案通常會儲存在 `/content/` 目錄下。

### 步驟 3: 取得檔案路徑
1.  在 Colab 檔案瀏覽器中，找到您已上傳的檔案。
2.  **在檔案名稱上點擊右鍵**。
3.  選擇菜單中的「**複製路徑**」(Copy path)。

### 步驟 4: 執行程式並輸入路徑
執行下方的程式碼區塊。當程式提示您輸入檔案路徑時，請將您在步驟 3 複製到的正確路徑貼上。

例如:
*   如果您的 SAP.txt 上傳後路徑是 `/content/SAP.txt`，就輸入 `/content/SAP.txt`。
*   如果您的 1319D.csv 上傳後路徑是 `/content/1319D.csv`，就輸入 `/content/1319D.csv`。


In [ ]:
compare_boms()